In [4]:
from google.cloud import bigquery
from google.cloud import bigquery_storage

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/rsfagundes/Documents/keys/segment-generator-e51ee8f89983.json"


In [5]:
client = bigquery.Client()
bqstorage = bigquery_storage.BigQueryReadClient()

E0000 00:00:1758989750.980892   51035 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [7]:
query = """
WITH
  first_touch AS (
  SELECT
    user_id,
    traffic_source AS first_touch_source,
    SUM(revenue) OVER (PARTITION BY user_id ) AS user_total_revenue
  FROM
    eduki_test.marketing_analytics
  QUALIFY
    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY purchase_date ) = 1 )
SELECT
  first_touch_source,
  ROUND(SUM(user_total_revenue), 2) AS total_revenue,
  ROUND( 100 * SUM(user_total_revenue) / SUM(SUM(user_total_revenue)) OVER (), 2 ) AS revenue_share
FROM
  first_touch
GROUP BY
  first_touch_source
ORDER BY
  total_revenue DESC;
"""

df = client.query(query).to_dataframe(bqstorage_client=bqstorage)

df.head()

,first_touch_source,total_revenue,revenue_share
0,Paid Search,148772.73,17.90
1,Direct,141955.49,17.08
2,Email,141477.52,17.03
3,Referral,136105.47,16.38
4,Social Media,134015.42,16.13


In [8]:
query = """
WITH
  first_rows AS (
  SELECT
    user_id,
    purchase_date,
    LEAD(purchase_date) OVER (PARTITION BY user_id ORDER BY purchase_date) AS next_purchase
  FROM
    `eduki_test.marketing_analytics`
  QUALIFY
    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY purchase_date) = 1 )
SELECT
  ROUND(100 * COUNTIF(TIMESTAMP_DIFF(next_purchase, purchase_date, DAY) <= 30) / COUNT(*), 2) AS pct_second_purchase
FROM
  first_rows;
"""

df = client.query(query).to_dataframe(bqstorage_client=bqstorage)

df.head()

,pct_second_purchase
0,18.51


In [9]:
query = """
SELECT
  campaign_id,
  campaign_name,
  ROUND(SUM(ad_spent),2) AS ad_spent,
  ROUND(SUM(revenue),2) AS revenue,
  ROUND(SAFE_DIVIDE(SUM(revenue),SUM(ad_spent)),2) AS ROAS,
  CASE
    WHEN SAFE_DIVIDE(SUM(revenue),SUM(ad_spent)) < 2 THEN 'Optmize or Pause'
    ELSE 'Performing well'
END
  AS Recomendation
FROM
  `eduki_test.marketing_analytics`
WHERE
  campaign_name IS NOT NULL
GROUP BY
  campaign_id,
  campaign_name
ORDER BY
  ROAS DESC;
"""

df = client.query(query).to_dataframe(bqstorage_client=bqstorage)

df.head()

,campaign_id,campaign_name,ad_spent,revenue,ROAS,Recomendation
0,101,Google Ads - Spring Sale,28297.52,76971.18,2.72,Performing well
1,202,Instagram - Summer Deal,25303.84,68617.28,2.71,Performing well
2,201,Facebook - Holiday Promo,42161.41,61287.46,1.45,Optmize or Pause
3,302,VIP Email - Exclusive Discount,47631.24,68170.07,1.43,Optmize or Pause
4,301,Newsletter - Monthly Offer,98896.04,67744.99,0.69,Optmize or Pause
